In [ ]:
# WARNING - CONCEPTUALLY QUESTIONABLE, DATA IS ACTUALLY BERNOUILLIE DISTRIBUTION

In [6]:
import numpy as np
import pandas as pd
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB

def build_confusion_matrix(dataset, model):
        X_train, X_test, y_train, y_test = train_test_split(dataset.iloc[:, :-1], dataset.iloc[:, -1], test_size=0.25)
        predictions = model.fit(X_train, y_train).predict(X_test)
        tn, fn, fp, tp = confusion_matrix(predictions, y_test.values).ravel()
        return {'true_negative': tn, 'false_negative': fn, 'false_positive': fp, 'true_positive': tp}

def sensitivity(cm):
    return cm['true_positive'] / (cm['true_positive'] + cm['false_negative'])

def specificity(cm):
    return cm['true_negative'] / ( cm['true_negative'] + cm['false_positive'] )

def false_positive_rate(cm):
    return (1.0 - (specificity(cm)) )

def generate_scores(dataset, model):
    specificity_scores = []
    sensitivity_scores = []

    for n in range(1000):
        cm = build_confusion_matrix(dataset, model)
        specificity_scores.append(specificity(cm))
        sensitivity_scores.append(sensitivity(cm))
    
    spec_avg = pd.Series(specificity_scores).mean()
    sens_avg = pd.Series(sensitivity_scores).mean()
    
    return {'specificity': spec_avg, 'sensitivity': sens_avg }

def summarise_test(test):
    x ='For {}, in model {}\nAverage NO SIGFALL accuracy is {}\nAverage SIGFALL accuracy is {}\n---\n'
    print(x.format(test['data_name'],test['model'],test['specificity'],test['sensitivity']))    


In [4]:
# Load data sets
inc_updrs_sigfall = pd.read_csv('./working_data/normalised_increase_updrs_sigfall.csv')
inc_updrs_sigfall_raw = pd.read_csv('./working_data/normalised_increase_updrs_sigfall_raw.csv')

datasets = [{'data':inc_updrs_sigfall, 'name':'inc_updrs_sigfall'}, 
            {'data':inc_updrs_sigfall_raw, 'name':'inc_updrs_sigfall_raw'}]

In [8]:
# Run tests to score each model & dataset combination

models = [
    MultinomialNB(alpha=1, fit_prior=True),
    MultinomialNB(alpha=0.75, fit_prior=True),
    MultinomialNB(alpha=0.5, fit_prior=True),
    MultinomialNB(alpha=0.25, fit_prior=True),
    MultinomialNB(alpha=0.1, fit_prior=True),
    MultinomialNB(alpha=1, fit_prior=False),
    MultinomialNB(alpha=0.75, fit_prior=False),
    MultinomialNB(alpha=0.5, fit_prior=False),
    MultinomialNB(alpha=0.25, fit_prior=False),
    MultinomialNB(alpha=0.1, fit_prior=False),
]

tests = []
for m in models:
    for d in datasets:
        tests.append({
            'data_name': d['name'],
            'dataset': d['data'],
            'model': m
        })

for t in tests:
    s = generate_scores(dataset=t['dataset'], model=t['model'])
    t['specificity'] = s['specificity']
    t['sensitivity'] = s['sensitivity']
    summarise_test(t)

For inc_updrs_sigfall, in model MultinomialNB(alpha=1)
Average NO SIGFALL accuracy is 1.0
Average SIGFALL accuracy is 0.0
---

For inc_updrs_sigfall_raw, in model MultinomialNB(alpha=1)
Average NO SIGFALL accuracy is 1.0
Average SIGFALL accuracy is 0.0
---

For inc_updrs_sigfall, in model MultinomialNB(alpha=0.75)
Average NO SIGFALL accuracy is 1.0
Average SIGFALL accuracy is 0.0
---

For inc_updrs_sigfall_raw, in model MultinomialNB(alpha=0.75)
Average NO SIGFALL accuracy is 1.0
Average SIGFALL accuracy is 0.0
---

For inc_updrs_sigfall, in model MultinomialNB(alpha=0.5)
Average NO SIGFALL accuracy is 1.0
Average SIGFALL accuracy is 0.0
---

For inc_updrs_sigfall_raw, in model MultinomialNB(alpha=0.5)
Average NO SIGFALL accuracy is 1.0
Average SIGFALL accuracy is 0.0
---

For inc_updrs_sigfall, in model MultinomialNB(alpha=0.25)
Average NO SIGFALL accuracy is 1.0
Average SIGFALL accuracy is 0.0
---

For inc_updrs_sigfall_raw, in model MultinomialNB(alpha=0.25)
Average NO SIGFALL accur

In [11]:
# The smoothing function appears to have little affect
# Priors from data prevent any prediction of SIGFALL due to imbalance i the data set

# Can we define false priors to increase SIGFALL accuracy? -> No, overfits to the opposite problem, everything is SIGFALL

fp_models = [
    MultinomialNB(alpha=1, fit_prior=False, class_prior=[0.1,0.9]),
    MultinomialNB(alpha=1, fit_prior=False, class_prior=[0.2,0.8]),
    MultinomialNB(alpha=1, fit_prior=False, class_prior=[0.3,0.7]),
    MultinomialNB(alpha=1, fit_prior=False, class_prior=[0.4,0.6]),
    MultinomialNB(alpha=1, fit_prior=False, class_prior=[0.5,0.5]),
]

fp_tests = []
for m in fp_models:
    for d in datasets:
        fp_tests.append({
            'data_name': d['name'],
            'dataset': d['data'],
            'model': m
        })

for t in fp_tests:
    s = generate_scores(dataset=t['dataset'], model=t['model'])
    t['specificity'] = s['specificity']
    t['sensitivity'] = s['sensitivity']
    summarise_test(t)


For inc_updrs_sigfall, in model MultinomialNB(alpha=1, class_prior=[0.1, 0.9], fit_prior=False)
Average NO SIGFALL accuracy is 0.0
Average SIGFALL accuracy is 1.0
---

For inc_updrs_sigfall_raw, in model MultinomialNB(alpha=1, class_prior=[0.1, 0.9], fit_prior=False)
Average NO SIGFALL accuracy is 0.0
Average SIGFALL accuracy is 1.0
---

For inc_updrs_sigfall, in model MultinomialNB(alpha=1, class_prior=[0.2, 0.8], fit_prior=False)
Average NO SIGFALL accuracy is 0.0
Average SIGFALL accuracy is 1.0
---

For inc_updrs_sigfall_raw, in model MultinomialNB(alpha=1, class_prior=[0.2, 0.8], fit_prior=False)
Average NO SIGFALL accuracy is 0.0
Average SIGFALL accuracy is 1.0
---

For inc_updrs_sigfall, in model MultinomialNB(alpha=1, class_prior=[0.3, 0.7], fit_prior=False)
Average NO SIGFALL accuracy is 0.0
Average SIGFALL accuracy is 1.0
---

For inc_updrs_sigfall_raw, in model MultinomialNB(alpha=1, class_prior=[0.3, 0.7], fit_prior=False)
Average NO SIGFALL accuracy is 0.0
Average SIGFALL a

In [13]:
#  Lets try Complementary Bayes
from sklearn.naive_bayes import ComplementNB

cp_models = [
    ComplementNB(alpha=1, norm=False),
    ComplementNB(alpha=1, norm=True),
]

cp_tests = []
for m in cp_models:
    for d in datasets:
        cp_tests.append({
            'data_name': d['name'],
            'dataset': d['data'],
            'model': m
        })

for t in cp_tests:
    s = generate_scores(dataset=t['dataset'], model=t['model'])
    t['specificity'] = s['specificity']
    t['sensitivity'] = s['sensitivity']
    summarise_test(t)

For inc_updrs_sigfall, in model ComplementNB(alpha=1)
Average NO SIGFALL accuracy is 0.7789998413791391
Average SIGFALL accuracy is 0.42864181791230144
---

For inc_updrs_sigfall_raw, in model ComplementNB(alpha=1)
Average NO SIGFALL accuracy is 0.7790445431749553
Average SIGFALL accuracy is 0.4323141953516266
---

For inc_updrs_sigfall, in model ComplementNB(alpha=1, norm=True)
Average NO SIGFALL accuracy is 0.7813339545727074
Average SIGFALL accuracy is 0.4285750033407668
---

For inc_updrs_sigfall_raw, in model ComplementNB(alpha=1, norm=True)
Average NO SIGFALL accuracy is 0.7791482582547407
Average SIGFALL accuracy is 0.4320431995156033
---

